In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import json

In [42]:
#load the dataset and separate the features from the labels
df=pd.read_csv('dataset.csv')
X=np.array(df[["version","ihl","tos","total_length","identification","flags","fragment_offset","ttl","protocol","header_checksum","source_ip","dest_ip"]],dtype=np.chararray)
Y=np.array(df[["label"]],dtype=np.int8)
X=X.reshape(X.shape[0],X.shape[1])
Y=Y.reshape(Y.shape[0],)

#split the dataset into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)




In [58]:
#create the model for the decision tree
Model=DecisionTreeClassifier(criterion='entropy',max_depth=10,splitter='best')
#train the model
Model.fit(X_train,Y_train)
#predict the labels for the testing dataset
y_pred=Model.predict(X_test)
#calculate the accuracy of the model
accuracy=sklearn.metrics.accuracy_score(Y_test,y_pred)

print(Model.score(X_test,Y_test))
#save Model to file

def convert_arrays(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, DecisionTreeClassifier):
        return obj.__getstate__()
    else:
        raise TypeError(f"Type {type(obj)} not serializable")

model_json = json.dumps(Model, default=convert_arrays)
filename = 'decision_tree_model.json'
with open(filename, 'w') as f:
    json.dump(json_model, f)





0.9872685185185185


TypeError: Type <class 'numpy.int64'> not serializable